In [ ]:
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

# OpenAI
client = OpenAI(api_key="")

# Pinecone
pc = Pinecone(api_key="")
index_name = "parametros"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

index = pc.Index(index_name)

In [ ]:
import json

# Carregar JSON com os dados
with open(r'output_paginas\paginas_para_embeddings.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)

# Extrair os textos (chunks)
chunks = [item["texto"] for item in dados]

# Gerar embeddings
response = client.embeddings.create(
    input=chunks,
    model="text-embedding-3-small"
)

embeddings = response.data

# Preparar para upsert
to_upsert = [
    (
        str(dados[i]["pagina"]),  # ID do vetor
        embedding.embedding,       # Vetor de embedding
        {
            "text": chunks[i],     
            "pagina": dados[i]["pagina"],
            "tokens": dados[i].get("tokens", 0)
        }
    )
    for i, embedding in enumerate(embeddings)
]

# Up no Pinecone
index.upsert(vectors=to_upsert, namespace="parametros")

print("Embeddings enviados para o Pinecone com sucesso!")

In [3]:
def rag_manual_chat(query, openai_client, index, threshold=0.1, top_k=3):
    
    # Gerar embedding da query
    query_embedding = openai_client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    ).data[0].embedding

    # Buscar no Pinecone
    response = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        namespace="parametros"
    )

    # Filtrar pelo threshold
    matches = [m for m in response.matches if m.score >= threshold]

    if not matches:
        return "Desculpe, não encontrei informações relevantes no manual."

    # Montar contexto
    contexto = "\n\n".join(
        [f"[Página {m.metadata.get('pagina', 'N/A')}]\n{m.metadata.get('text', '')}" for m in matches]
    )

    # Prompt estruturado
    prompt = f"""
    Você é um assistente técnico especializado em portas industriais automáticas.
    Utilize **apenas as informações do manual abaixo** para responder à pergunta de forma precisa, técnica e completa.
    Se a resposta não estiver claramente no manual, diga que a informação não foi encontrada.

    Manual extraído:
    {contexto}

    Pergunta: {query}

    Responda:
    """

    # Gerar resposta com o modelo
    completion = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Você responde exclusivamente com base no manual técnico fornecido."},
            {"role": "user", "content": prompt}
        ],
        temperature=0  # resposta objetiva, sem criatividade
    )

    resposta = completion.choices[0].message.content.strip()

    return resposta

In [7]:
query = "parametros sobre freio"
response = rag_manual_chat(query, client, index)
print("Resposta:", response)


Resposta: Os parâmetros relacionados ao freio, conforme o manual, são os seguintes:

1. **P.366 s] - Duration of brake ramp “r8”**: Define o tempo da rampa de frenagem "r8" em milissegundos, variando de 15 a 500 ms. Este parâmetro é visível e ajustável apenas se o parâmetro P.216 estiver definido para 0.

2. **P.367 [Hz/s] - Acceleration of brake ramp “r8”**: Define a aceleração durante a rampa de frenagem "r8" em Hertz por segundo, variando de 5 a 300 Hz/s. Este parâmetro é visível e ajustável apenas se o parâmetro P.216 for maior que 0.

3. **P.187 - Contact type of brake relay**: Permite escolher o tipo de contato do relé de freio, com opções de contato normalmente aberto (N.O.) ou normalmente fechado (N.C.).

4. **P.18F - Brake profile**: Define o perfil de comportamento do freio. Os detalhes exatos deste perfil estão no apêndice do manual.

5. **P.189 [%] - Brake for lifting anchor motors**: Ajusta a tensão necessária para motores de ancoragem de elevação, expressa em porcentagem 

In [ ]:
# Example usage
while True:
  query = input()
  response = rag_manual_chat(query, client, index)
  print(f"User: {query}")
  print(f"Bot: {response}")